# Melliguer controller

## Setup and imports

In [1]:
import time
import numpy as np
import mujoco
from mujoco import viewer


# ============================================================
# 1) Utilidades matemáticas (simples)
# ============================================================

def clamp(x, lo, hi):
    """Limita un escalar entre [lo, hi]."""
    return max(lo, min(hi, x))

def vee(M):
    """
    Operador vee para matrices skew-symmetric:
    [  0  -z   y
       z   0  -x
      -y   x   0 ]  -> [x, y, z]
    """
    return np.array([M[2, 1], M[0, 2], M[1, 0]])

def quat_to_rotmat(q):
    """Quaternion (w,x,y,z) -> matriz de rotación 3x3."""
    w, x, y, z = q
    R = np.array([
        [1 - 2*(y*y + z*z),     2*(x*y - z*w),     2*(x*z + y*w)],
        [    2*(x*y + z*w), 1 - 2*(x*x + z*z),     2*(y*z - x*w)],
        [    2*(x*z - y*w),     2*(y*z + x*w), 1 - 2*(x*x + y*y)]
    ])
    return R


# ============================================================
# 2) Disturbancia: fuerza desconocida f^o (depende de posición)
# ============================================================

def unknown_force_world(pos):
    """
    Fuerza externa "desconocida" en coordenadas mundo.
    Puedes cambiarla por cualquier función suave de la posición.
    Unidades: Newtons (aprox).
    """
    x, y, z = pos
    fx = 0.8 * np.sin(0.7 * x)
    fy = 0.8 * np.cos(0.7 * y)
    fz = 0.4 * np.sin(0.5 * z)
    # return np.array([fx, fy, fz])
    return np.array([0.0, 0.0, 0.0])  # solo en XY para ver efecto horizontal
    


# ============================================================
# 3) Trayectoria deseada (por ahora: hover con pequeña oscilación)
# ============================================================

def desired_trajectory(t, p0):
    """
    Devuelve (p_d, v_d, a_d, yaw_d, yawdot_d).
    - p0: posición inicial para hover.
    """
    # Pequeña oscilación para que el control "se vea"
    # p_d = p0 + np.array([0.2*np.sin(0.3*t), 0.2*np.cos(0.3*t), 0.0])
    # v_d = np.array([0.2*0.3*np.cos(0.3*t), -0.2*0.3*np.sin(0.3*t), 0.0])
    # a_d = np.array([-0.2*(0.3**2)*np.sin(0.3*t), -0.2*(0.3**2)*np.cos(0.3*t), 0.0])

    # p_d = p0 + np.array([0.2*np.sin(0.3*t), 0.0*np.cos(0.3*t), 0.2*np.cos(0.3*t)])
    # v_d = np.array([0.2*0.3*np.cos(0.3*t), -0.0*0.3*np.sin(0.3*t), -0.2*0.3*np.sin(0.3*t)])
    # a_d = np.array([-0.2*(0.3**2)*np.sin(0.3*t), -0.0*(0.3**2)*np.cos(0.3*t), -0.2*(0.3**2)*np.cos(0.3*t)])

    p_d = p0 + np.array([-1.7*np.sin(0.3*t), 0.0, -0.5])
    v_d = np.array([-1.7*0.3*np.cos(0.3*t), 0.0, -0.0])
    a_d = np.array([1.7*(0.3**2)*np.sin(0.3*t), 0.0, -0.0])

    yaw_d = 0.0
    yawdot_d = 0.0
    return p_d, v_d, a_d, yaw_d, yawdot_d


# ============================================================
# 4) Controlador Mellinger (versión clara y práctica)
# ============================================================

def mellinger_control(pos, vel, R, omega_body, p_d, v_d, a_d, yaw_d, yawdot_d, m, J):
    """
    Implementación simple del controlador tipo Mellinger:
      - Control de posición -> fuerza deseada F_des (mundo)
      - Construir R_des desde F_des y yaw_d
      - Control de actitud -> torque deseado M (en body)
      - Salida: thrust_total (N) y momentos M (N*m)

    NOTA:
    - omega_body debe estar en el marco body.
    - J aquí lo tomamos diagonal para simplificar.
    """

    # ----------------------------
    # Ganancias (tuneables)
    # ----------------------------
    Kp = np.diag([4.0, 4.0, 8.0])     # posición
    Kv = np.diag([3.0, 3.0, 5.0])     # velocidad

    KR = np.diag([8.0, 8.0, 2.0])     # actitud
    Kw = np.diag([0.25, 0.25, 0.15])  # velocidad angular

    g = 9.81
    e3 = np.array([0.0, 0.0, 1.0])

    # ----------------------------
    # (A) Control de posición -> aceleración deseada
    # e_p = p - p_d (convención común)
    # ----------------------------
    e_p = pos - p_d
    e_v = vel - v_d

    # a_cmd = a_d - Kp*e_p - Kv*e_v + g*e3
    a_cmd = a_d - Kp @ e_p - Kv @ e_v + g * e3

    # Fuerza deseada en el mundo
    F_des = m * a_cmd

    # ----------------------------
    # (B) Construir orientación deseada R_des
    # - b3_des apunta en dirección de F_des
    # - b1_des fija el yaw
    # ----------------------------
    F_norm = np.linalg.norm(F_des)
    if F_norm < 1e-6:
        F_norm = 1e-6

    b3_des = F_des / F_norm

    # Vector "frontal" según yaw deseado (en plano XY del mundo)
    b1_c = np.array([np.cos(yaw_d), np.sin(yaw_d), 0.0])

    # b2_des = b3 x b1_c (normalizado)
    b2_des = np.cross(b3_des, b1_c)
    if np.linalg.norm(b2_des) < 1e-6:
        b2_des = np.array([0.0, 1.0, 0.0])
    else:
        b2_des = b2_des / np.linalg.norm(b2_des)

    # b1_des = b2 x b3
    b1_des = np.cross(b2_des, b3_des)

    # Matriz deseada
    R_des = np.column_stack((b1_des, b2_des, b3_des))

    # ----------------------------
    # (C) Thrust total (N)
    # thrust = F_des · b3_actual
    # b3_actual = tercera columna de R
    # ----------------------------
    b3 = R[:, 2]
    thrust = float(F_des.dot(b3))

    # ----------------------------
    # (D) Error de actitud (forma geométrica)
    # e_R = 0.5 * vee(R_des^T R - R^T R_des)
    # ----------------------------
    e_R_mat = 0.5 * (R_des.T @ R - R.T @ R_des)
    e_R = vee(e_R_mat)

    # Velocidad angular deseada (simplificada)
    # Para empezar: solo yawdot alrededor de e3 en mundo -> lo pasamos a body aprox
    omega_des_world = np.array([0.0, 0.0, yawdot_d])
    omega_des_body = R.T @ omega_des_world

    e_omega = omega_body - omega_des_body

    # ----------------------------
    # (E) Torque deseado (en body)
    # M = -KR e_R - Kw e_omega + omega x (J omega)
    # ----------------------------
    Jomega = J @ omega_body
    coriolis = np.cross(omega_body, Jomega)

    M = -KR @ e_R - Kw @ e_omega + coriolis

    return thrust, M


# ============================================================
# 5) Mixer simple (thrust + torques -> 4 motores)
# ============================================================

def simple_mixer(thrust, M, ctrl_min=0.0, ctrl_max=2.0):
    """
    Mixer "práctico" para tu XML (4 motores con ctrlrange 0..2).
    OJO: esto NO usa parámetros reales (l, kf, km).
    Es un mezclador simple para que funcione en simulación.

    thrust: (N) lo escalamos a un valor de control
    M: torque en body (Mx, My, Mz) lo escalamos
    """

    # Escalas (tuneables según tu modelo/masa)
    kT = 0.6     # convierte Newton -> señal de motor (aprox)
    kMx = 0.8    # torque roll -> diferencia motores
    kMy = 0.8    # torque pitch
    kMz = 0.3    # torque yaw

    # Componente base igual para los 4 motores
    uT = kT * thrust

    Mx, My, Mz = M

    # Mezcla tipo "plus" (ajusta signos si tu dron gira raro)
    u0 = uT - kMy*My - kMz*Mz   # motor0
    u1 = uT + kMy*My + kMz*Mz   # motor1
    u2 = uT + kMx*Mx - kMz*Mz   # motor2
    u3 = uT - kMx*Mx + kMz*Mz   # motor3

    # Saturación al rango del XML
    ctrl = np.array([
        clamp(u0, ctrl_min, ctrl_max),
        clamp(u1, ctrl_min, ctrl_max),
        clamp(u2, ctrl_min, ctrl_max),
        clamp(u3, ctrl_min, ctrl_max),
    ], dtype=float)

    return ctrl


# ============================================================
# 6) Main: correr MuJoCo + control
# ============================================================

def main():
    # --- Cambia aquí tu XML ---
    XML_PATH = "models/jiawei_model_rep_single.xml"
    XML_PATH = "models/single_quadrotor.xml"

    # Cargar modelo
    model = mujoco.MjModel.from_xml_path(XML_PATH)
    data = mujoco.MjData(model)

    dt = model.opt.timestep

    # Obtener ID del body del dron
    drone_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, "drone_3d")

    # Masa total del dron (incluye subcuerpos)
    m = float(model.body_subtreemass[drone_id])

    # Inercia (simplificada): usamos diagonal del body
    # (en modelos complejos esto no es perfecto, pero sirve para empezar)
    J_diag = model.body_inertia[drone_id].copy()
    J = np.diag(J_diag)

    # Posición inicial como referencia de hover
    mujoco.mj_forward(model, data)
    p0 = data.xpos[drone_id].copy()

    # Abrir visor
    with viewer.launch_passive(model, data) as v:
        t = 0.0

        while v.is_running():
            # ------------------------------------------------
            # (1) Leer estado del dron desde MuJoCo
            # ------------------------------------------------
            pos = data.xpos[drone_id].copy()

            # cvel: [omega_world, v_world] en el marco mundo (aprox)
            omega_world = data.cvel[drone_id, 0:3].copy()
            vel_world   = data.cvel[drone_id, 3:6].copy()

            quat = data.xquat[drone_id].copy()
            R = quat_to_rotmat(quat)

            # Convertir omega a marco body (para control geométrico)
            omega_body = R.T @ omega_world

            # ------------------------------------------------
            # (2) Aplicar fuerza desconocida al dron (disturbancia)
            # ------------------------------------------------
            f_unk = unknown_force_world(pos)

            # xfrc_applied: [torque(3), force(3)] en mundo, aplicado al body
            data.xfrc_applied[drone_id, 0:3] = 0.0     # sin torque externo
            data.xfrc_applied[drone_id, 3:6] = f_unk   # fuerza externa

            # ------------------------------------------------
            # (3) Generar referencia deseada
            # ------------------------------------------------
            p_d, v_d, a_d, yaw_d, yawdot_d = desired_trajectory(t, p0)

            # ------------------------------------------------
            # (4) Control Mellinger -> thrust (N) y torque M (N*m)
            # ------------------------------------------------
            thrust, M = mellinger_control(
                pos=pos,
                vel=vel_world,
                R=R,
                omega_body=omega_body,
                p_d=p_d,
                v_d=v_d,
                a_d=a_d,
                yaw_d=yaw_d,
                yawdot_d=yawdot_d,
                m=m,
                J=J
            )

            # ------------------------------------------------
            # (5) Mixer -> comandos a 4 motores (0..2 según tu XML)
            # ------------------------------------------------
            ctrl = simple_mixer(thrust, M, ctrl_min=0.0, ctrl_max=2.0)
            data.ctrl[:] = ctrl

            # ------------------------------------------------
            # (6) Paso de simulación
            # ------------------------------------------------
            start = time.time()

            mujoco.mj_step(model, data)
            v.sync()
            t += dt

            # --- sincronizar con tiempo real ---
            elapsed = time.time() - start
            sleep_time = dt - elapsed
            if sleep_time > 0:
                time.sleep(sleep_time)



if __name__ == "__main__":
    main()
